In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
from sklearn.metrics import mean_absolute_error

In [2]:
ss = pd.read_csv('SampleSubmission.csv')
ss['country'] = [x[:-11].replace(' (the)', '') for x in ss['Territory X Date'].tolist()]

In [3]:
date = '4/19/20'
confirmed_data = pd.read_csv('COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_data.loc[:, date] = 0

In [4]:
for country in ['Australia', 'Canada', 'China']:
    tmp_data = confirmed_data[confirmed_data['Country/Region'] == country]
    sum_vals = tmp_data.values[:, 4:].sum(axis = 0).tolist()
    vals = tmp_data.values[0, :4].tolist() + sum_vals
    max_index = confirmed_data.index[-1]
    confirmed_data = confirmed_data.drop(tmp_data.index)
    confirmed_data.loc[max_index + 1, :] = [np.nan] + vals[1:]
    
confirmed_data['country'] = [s + '_' + c if s == s else c for s,c in confirmed_data[['Province/State', 'Country/Region']].values ]

In [5]:
dict_reaplce = {"Cote d'Ivoire": "Côte d'Ivoire", 'Russia': 'Russian Federation',  'Bolivia': "Bolivia (Plurinational State of)",
    'Brunei': 'Brunei Darussalam' , 'Cayman Islands_United Kingdom' : 'Cayman Islands', 'Bermuda_United Kingdom' : 'Bermuda', 
    'Aruba_Netherlands': 'Aruba',  'Taiwan*': 'Taiwan',  'US' : 'United States of America', "Greenland_Denmark" : 'Greenland',
    'Venezuela': 'Venezuela (Bolivarian Republic of)', 'Reunion_France' : 'Reunion', 'Vietnam' : 'Viet Nam',
    'New Caledonia_France' : 'New Caledonia', 'French Polynesia_France' : 'French Polynesia', 'French Guiana_France' : 'French Guiana',
    'Faroe Islands_Denmark' : 'Faroe Islands' ,'Iran' : 'Iran (Islamic Republic of)', 
    'Korea, South': "Republic of Korea", 'Moldova' : 'Republic of Moldova', 'Congo (Kinshasa)' : 'Democratic Republic of the Congo',
    'Congo (Brazzaville)' : 'Congo','United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
    'Tanzania' : 'United Republic of Tanzania', 'Syria': 'Syrian Arab Republic', 'St Martin_France' : 'St Martin',
    'Saint Barthelemy_France' : 'Saint Barthelemy', 'Montserrat_United Kingdom' : 'Montserrat', 'Mayotte_France': 'Mayotte',
    'Laos': "Lao People's Democratic Republic", 'Guadeloupe_France' : 'Guadeloupe', 'Curacao_Netherlands': 'Curacao'}

confirmed_data['country'] = [dict_reaplce.get(x,x) for x in confirmed_data['country'].unique()]

In [6]:
today_countries_state = pd.read_csv("COVID-19-new/data/cases_state.csv")
today_countries = pd.read_csv("COVID-19-new/data/cases_country.csv")

In [7]:
dict_today_vals = {}
col = 'Deaths'

In [8]:
today_countries_state['State_Country'] = [s + '_' + c if s == s else c for s,c in today_countries_state[['Province_State', 'Country_Region']].values ]
today_countries_state['State_Country'] = [dict_reaplce.get(x,x) for x in today_countries_state['State_Country'].unique()]
for country, val in today_countries_state[['State_Country', col]].values:
    dict_today_vals[country] = val

today_countries['State_Country'] = [dict_reaplce.get(x,x) for x in today_countries['Country_Region'].unique()]
for country, val in today_countries[['State_Country', col]].values:
    dict_today_vals[country] = val

In [9]:
diff_country = list(set(ss['country'].unique()) - set(confirmed_data['country'].unique()))

for country in diff_country:
    confirmed_data.loc[ confirmed_data.index[-1] + 1, :] = 0
    confirmed_data.iloc[-1, -1] = country
    
confirmed_data = confirmed_data.reset_index(drop = True)    

In [10]:
confirmed_data[date] = [dict_today_vals.get(c, 0) for c in confirmed_data['country'].values]
confirmed_data = confirmed_data[ confirmed_data['country'].isin(ss['country'].unique()) ].reset_index(drop = True)  

In [11]:
dict_values = {}
for val in confirmed_data.values:
    dict_values[val[-1]] = val[4:-1]
    
for country in dict_values:
    if sum(np.diff(dict_values[country]) < 0):
        print(country)
        new_val = [dict_values[country][-1]]
        for val_1, val_2 in zip(dict_values[country][1:][::-1], dict_values[country][:-1][::-1]):
            if val_2 <= new_val[-1]:
                new_val += [val_2]
            else:
                new_val += [new_val[-1]]
        new_val = np.array(new_val[::-1])
        ind_country = confirmed_data[confirmed_data['country'] == country].index[0]
        confirmed_data.iloc[ind_country, 4:-1] = new_val

Cyprus
Finland
Germany
Iceland
India
Kazakhstan
Luxembourg
Philippines
Serbia
Slovakia


In [12]:
def predict(data, country, len_predict, metrics, len_intersection, bound_0, bound_1):
    country_data = data[data['country']==country].values[0][4:-1]

    if metrics != 'Fatalities':
        if country_data.max() > 5000:
            start_people = 3
        elif country_data.max() > 500:
            start_people = 2
        else:
            start_people = 0
    else:
        if country_data.max() > 50:
            start_people = 1
        else:
            start_people = 0        

    country_data = country_data[dict_case_date[country][start_people]:]

    x_data = np.arange(len(country_data))
    y_data = np.array(country_data, dtype = float)

    if len(x_data) <= 1:
        y_val = np.arange(len(x_data), len(x_data) + len_predict)
        if metrics != 'Fatalities':
            return [-1, -1, -1], log_curve(y_val, bound_0, bound_1, 10, 1)
        else:
            return [-1, -1, -1], log_curve(y_val, bound_0, bound_1, 1, 0)            
    else:
        if metrics != 'Fatalities':
#             y_max = max(country_data['Population'].values[0] * 0.001, y_data[-1] * 5)
            y_max = y_data[-1] * 15
        else:
            y_max = y_data[-1] * 15
        y_min = y_data[-1]
        if metrics != 'Fatalities':
            diff_k = max(1,  y_data[-1] - y_data[-2])
        else:
            diff_k = 1

        best_score = 100000000000

        if len(x_data) < 10:
            range_num = 1
        else:
            range_num = len(x_data) - 9
            
        for i in range(range_num):
            popt, pcov = curve_fit(log_curve, x_data[i:], y_data[i:],  bounds=([bound_0 - 0.05, bound_1 - 7.5 , y_min, 0 ],
                                                                  [bound_0 + 0.1, bound_1 + 7.5, y_max, diff_k]),
                                p0=[bound_0, bound_1 ,(y_min + y_max) / 2, 0], maxfev=100000)
            
            tmp_pred = log_curve(x_data[-5:], popt[0], popt[1], popt[2], popt[3])
            score = rmsle(y_data[-5:], tmp_pred)
            if score < best_score:
                best_score = score
                best_i = i
                best_popt = popt
        

        y_val = np.arange(len(x_data) - len_intersection, len(x_data) + len_predict - len_intersection)

        return  popt, log_curve(y_val, best_popt[0], best_popt[1], best_popt[2], best_popt[3])
    


def log_curve(x, k, x_0, ymax, x_1):
    return ymax / (1 + np.exp(-k*(x-x_0))) + x_1 * x

def rmsle(true, pred):
    return mean_absolute_error(true,pred)

In [13]:
metrics = 'Fatalities'
# num_val_dates = 7
num_val_dates = 0
# val_range = np.arange(confirmed_data.shape[1]).tolist()[: -(num_val_dates + 1)] + [-1]
# data_train = confirmed_data.iloc[:, val_range].copy()
data_train = confirmed_data.iloc[:, :].copy()

dict_values = {}
for val in data_train.values:
    dict_values[val[-1]] = val[4:-1]

dict_case_date = {}
for country in dict_values:
    dict_case_date[country] = []
    for case in [1, 5]:
        try:
            dict_case_date[country] += [np.where(dict_values[country] >= case)[0][0]]
        except:
            dict_case_date[country] += [-1]
    dict_case_date[country] = np.array(dict_case_date[country])
    
len_pred = ss[ss.country == 'Albania'].shape[0]
len_intersection = confirmed_data.columns[48:-1].shape[0]

In [14]:
# val_range = [0,1,2,3] + np.arange(confirmed_data.shape[1]).tolist()[-(num_val_dates + 1):]
# data_val = confirmed_data.iloc[:, val_range].copy()

dict_predict = {}

for country in tqdm(data_train['country'].unique()):

    popt, pred_values = predict(data_train, country, len_pred, metrics,len_intersection, 0.15, 30)
    last_val = data_train[data_train['country'] == country].values[0][-2]
    if last_val > pred_values[len_intersection]:
        pred_values += np.abs(last_val - pred_values[len_intersection])
    dict_predict[country] = pred_values
    
# pred_list = []
# true_list = []
# for country in data_train['country'].unique()[ :50]:
#     pred_list += dict_predict[country].tolist()[len_intersection  : len_intersection + num_val_dates]
#     true_list += data_val[data_val['country'] == country].values[0][4: - 1].tolist()
    
# print(rmsle(true_list, pred_list))

ss['Death'] = 0
for country in ss['country'].unique():
    ss.loc[ss['country'] == country, 'Death'] = dict_predict[country].tolist()

C:\Users\Sergei\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:787: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [15]:
def simple_pred(vals, len_intersection, len_predict):
    diff_vals = (vals[-1] - vals[-4] ) / 3 * 0.95
    answer = np.ones(len_predict)
    answer[:len_intersection] = vals[-len_intersection : ]
    start_val = vals[- 1]
    for i in range(len_predict - len_intersection):
        start_val += diff_vals
        answer[len_intersection + i] = start_val
    return answer

dict_predict_new = {}

for country in tqdm(data_train['country'].unique()):
    vals = data_train[data_train['country'] == country].values[0][-len_intersection - 1: - 1]
    pred_values = simple_pred(vals, len_intersection, len_pred)
    dict_predict_new[country] = pred_values
    
# pred_list = []
# true_list = []
# for country in data_train['country'].unique()[ :50]:
#     pred_list += dict_predict_new[country].tolist()[len_intersection : len_intersection + num_val_dates]
#     true_list += data_val[data_val['country'] == country].values[0][4: - 1].tolist()
    
# print(rmsle(true_list, pred_list))


ss['New_Death'] = 0
for country in ss['country'].unique():
    ss.loc[ss['country'] == country, 'New_Death'] = dict_predict_new[country].tolist()

In [16]:
ss_confirm = pd.read_csv('submision_cc.csv')

In [17]:
ss['ConfirmedCases'] = ss_confirm['ConfirmedCases']
ss['New_ConfirmedCases'] = ss_confirm['New_ConfirmedCases']

In [18]:
k = 0.4

In [19]:
ss['mean_ConfirmedCases'] = (ss['ConfirmedCases'] * k + ss['New_ConfirmedCases'] * (1 - k)) 

In [20]:
ss['target'] = (ss['Death'] * k + ss['New_Death'] * (1 - k))
ss['target'] = ss['target'].clip(0, ss['mean_ConfirmedCases'] * 0.2)

In [23]:
ss['Territory X Date'] = [(x[:-8] + x[-7:]).replace('/0', '/') for x in ss['Territory X Date'].values]

In [25]:
ss[['Territory X Date', 'target']].to_csv('predict_{}.csv'.format(k), index = False)